# Generative Adversarial Network

In this notebook, we'll be building a generative adversarial network (GAN) trained on the network flow dataset.

In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload
import os

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.8.0
Eager execution: True


## Load and normalize data

In [12]:
import utils, models
reload(utils)
reload(models)
from models import GeneratorModel, DiscriminatorModel
from utils import max_norm, parse_feature_label, train_test_split, loss, grad, sample_n_number

In [3]:
# define some constants
IDS_DATASET = os.path.join('data', 'ids2017_sampled.csv')
RELEVANT_FEATURES = [' Source Port', ' Destination Port', ' Flow Duration', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', 'Bwd Packet Length Max', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Packets/s', ' Packet Length Mean', ' ACK Flag Count', ' Down/Up Ratio', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Bwd Avg Bytes/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', ' Active Std', ' Active Min', ' Idle Max']
LABEL_NAME = ' Label'
BENIGN_LABEL = 0
ATTACK_LABEL = 2
TRAIN_FRAC = 0.3
FEATURE_NUM_MODIFIED = 2

In [4]:
# quick view of dataset
!head -n5 {IDS_DATASET}

Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp, Flow Duration, Total Fwd Packets, Total Backward Packets,Total Length of Fwd Packets, Total Length of Bwd Packets, Fwd Packet Length Max, Fwd Packet Length Min, Fwd Packet Length Mean, Fwd Packet Length Std,Bwd Packet Length Max, Bwd Packet Length Min, Bwd Packet Length Mean, Bwd Packet Length Std,Flow Bytes/s, Flow Packets/s, Flow IAT Mean, Flow IAT Std, Flow IAT Max, Flow IAT Min,Fwd IAT Total, Fwd IAT Mean, Fwd IAT Std, Fwd IAT Max, Fwd IAT Min,Bwd IAT Total, Bwd IAT Mean, Bwd IAT Std, Bwd IAT Max, Bwd IAT Min,Fwd PSH Flags, Bwd PSH Flags, Fwd URG Flags, Bwd URG Flags, Fwd Header Length, Bwd Header Length,Fwd Packets/s, Bwd Packets/s, Min Packet Length, Max Packet Length, Packet Length Mean, Packet Length Std, Packet Length Variance,FIN Flag Count, SYN Flag Count, RST Flag Count, PSH Flag Count, ACK Flag Count, URG Flag Count, CWE Flag Count, ECE Flag Count, Down/Up Ratio, Average Packet Size, Av

In [5]:
# read csv
df = pd.read_csv(IDS_DATASET)

In [6]:
# extract relevant features and label name
df = df[RELEVANT_FEATURES + [LABEL_NAME]]

In [7]:
# extract bengin and attack flows we want
benign_df, attack_df = df[(df[LABEL_NAME] == BENIGN_LABEL)], df[(df[LABEL_NAME] == ATTACK_LABEL)]

In [8]:
# rewrite label values
benign_df.loc[:, LABEL_NAME] = 0
attack_df.loc[:, LABEL_NAME] = 1

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# convert to numpy
benign, attack = benign_df.values, attack_df.values

In [10]:
# max normalization
benign[:, :len(RELEVANT_FEATURES)] = max_norm(benign[:, :len(RELEVANT_FEATURES)])
attack[:, :len(RELEVANT_FEATURES)] = max_norm(attack[:, :len(RELEVANT_FEATURES)])

In [13]:
# do train, test split separately on benign and attack
benign_train, benign_test = train_test_split(benign, train_size=TRAIN_FRAC)
attack_train, attack_test = train_test_split(attack, train_size=TRAIN_FRAC)

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
# concat to get training and testing data
train_np, test_np = np.concatenate([benign_train, attack_train]), np.concatenate([benign_test, attack_test])

## Generate Dataset

In [15]:
# convert to dataset
train_dataset, test_dataset = tf.data.Dataset.from_tensor_slices(train_np), tf.data.Dataset.from_tensor_slices(test_np)

train_dataset = train_dataset.map(parse_feature_label)
train_dataset = train_dataset.shuffle(buffer_size=train_np.shape[0] * 5)  # randomize
train_dataset = train_dataset.batch(100) # make batch

# View a single example entry from a batch
features, label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor(
[9.54503831e-01 8.26536500e-04 5.09829282e-04 2.95782147e-04
 3.36898396e-06 1.61815068e-02 3.25301205e-01 1.19560013e-02
 9.54321358e-04 0.00000000e+00 5.18406775e-04 9.54321358e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.50946146e-05
 1.63476157e-05 7.03479576e-02 0.00000000e+00 1.42857143e-01
 1.94049159e-02 8.50946146e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.95782147e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00], shape=(41,), dtype=float64)
example label: tf.Tensor(0, shape=(), dtype=int32)


## Build Model

In [16]:
g_model = GeneratorModel()
d_model = DiscriminatorModel()

## Build Optimizer

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

## Train

In [21]:
## Note: Rerunning this cell uses the same model variables

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 1

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()

    # Training loop - using batches of 32
#     for x, y in train_dataset:
#         if y == 0:
#             # Optimize the model
#             grads = grad(model, x, y)
#             optimizer.apply_gradients(
#                 zip(grads, model.variables),
#                 global_step=tf.train.get_or_create_global_step()
#             )
#         elif y == 1:
#             feature_nums = sample_n_number(len(RELEVANT_FEATURES), FEATURE_NUM_MODIFIED)
#             z = x[:, feature_nums]
#             modified_x = 
#         else:
#             raise AssertionError()

#         # Track progress
#         epoch_loss_avg(loss(model, x, y))  # add current batch loss
#         # compare predicted label to actual label
#         epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

#     # end epoch
#     train_loss_results.append(epoch_loss_avg.result())
#     train_accuracy_results.append(epoch_accuracy.result())

#     if epoch % 50 == 0:
#         print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
#                 epoch,
#                 epoch_loss_avg.result(),
#                 epoch_accuracy.result()
#             )
#         )

## Hyperparameters

In [ ]:
# Size of input flow to discriminator
input_size = benign_train.shape[1]
# Size of latent vector to generator
z_size = 2
# Sizes of hidden layers in generator and discriminator
g_hidden_size = [128, 64]
d_hidden_size = [128, 64]
# Leak factor for leaky ReLU
alpha = 0.01
# log interval
log_interval = 5

## Build network

Now we're building the network from the functions defined above.

First is to get our inputs, `input_benign, input_z, input_attack_remains` from `model_inputs` using the sizes of the input and z.

Then, we'll create the generator, `generator(input_z, z_size)`. This builds the generator with the appropriate input and output sizes.

Then the discriminators. We'll build two of them, one for benign flow data and one for attack flow data. Since we want the weights to be the same for both benign and attack flow data, we need to reuse the variables. For the attack flow data, we're getting it from the output of the generator concatenated with remaining part of attack feature called `g_model`. So the benign data discriminator is `discriminator(input_benign)` while the attack discriminator is `discriminator(g_model, reuse=True)`.

In [ ]:
tf.reset_default_graph()

# Create our input placeholders
input_benign, input_z, input_attack_remains = model_inputs(input_size, z_size, attack_remains_dim=input_size - z_size)

# Build the model
z_generated, g_hidden = generator(input_z, z_size, n_units=[z_size] + g_hidden_size, alpha=alpha)
g_model = tf.concat([z_generated, input_attack_remains], 1)
# g_model is the generator output concatenated with the remaining part of attack features

d_model_benign, d_logits_benign, d_hidden_benign = discriminator(input_benign,
                                                                 n_units= [input_size] + d_hidden_size,
                                                                 alpha=alpha)
d_model_attack, d_logits_attack, d_hidden_attack = discriminator(g_model,
                                                                 reuse=True,
                                                                 n_units=[input_size] + d_hidden_size,
                                                                 alpha=alpha)

## Discriminator and Generator Losses

Now we need to calculate the losses, which is a little tricky. For the discriminator, the total loss is the sum of the losses for benign and attack flows, `d_loss = d_loss_benign + d_loss_attack`. The losses will by sigmoid cross-entropys, which we can get with `tf.nn.sigmoid_cross_entropy_with_logits`. We'll also wrap that in `tf.reduce_mean` to get the mean for all the flows in the batch. So the losses will look something like 

```python
tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
```

For the benign flow logits, we'll use `d_logits_benign` which we got from the discriminator in the cell above. For the labels, we want them to be all zeros, since these are all benign flows. In TensorFlow, it looks something like `labels = tf.zeros_like(tensor)`

The discriminator loss for the attack flow data is similar. The logits are `d_logits_attack`, which we got from passing the generator output concatenated with remaing part of attack flow features to the discriminator. These attack logits are used with labels of all ones. Remember that we want the discriminator to output 0 for benign flows and 1 for attack flows, so we need to set up the losses to reflect that.

Finally, the generator losses are using `d_logits_attack`, the attack flow logits. But, now the labels are all zeros. The generator is trying to fool the discriminator, so it wants to discriminator to output zeros for attack flows.

In [ ]:
# Calculate losses
d_loss_benign = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_benign, 
                                                          labels=tf.zeros_like(d_logits_benign)))
d_loss_attack = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_attack, 
                                                          labels=tf.ones_like(d_logits_attack)))
d_loss = d_loss_benign + d_loss_attack

g_loss = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_attack,
                                                     labels=tf.zeros_like(d_logits_attack)))

## Optimizers

We want to update the generator and discriminator variables separately. So we need to get the variables for each part build optimizers for the two parts. To get all the trainable variables, we use `tf.trainable_variables()`. This creates a list of all the variables we've defined in our graph.

For the generator optimizer, we only want to generator variables. Our past selves were nice and used a variable scope to start all of our generator variable names with `generator`. So, we just need to iterate through the list from `tf.trainable_variables()` and keep variables to start with `generator`. Each variable object has an attribute `name` which holds the name of the variable as a string (`var.name == 'weights_0'` for instance). 

We can do something similar with the discriminator. All the variables in the discriminator start with `discriminator`.

Then, in the optimizer we pass the variable lists to `var_list` in the `minimize` method. This tells the optimizer to only update the listed variables. Something like `tf.train.AdamOptimizer().minimize(loss, var_list=var_list)` will only train the variables in `var_list`.

In [ ]:
# Optimizers
learning_rate = 0.002

# Get the trainable_variables, split into G and D parts
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

## Training

In [ ]:
from copy import deepcopy
import time

batch_size = 100
epochs = 1000
losses = []
original_flows, generated_flows, attack_scores = [], [], []
# Only save generator variables
saver = tf.train.Saver(var_list=g_vars)
# TODO: add outer-loop, which is test number, to see convergence trends
# TODO: modify training process - choose random 2 features and give 2 outputs randomly back
with tf.Session() as sess:
    total_start = time.time()
    sess.run(tf.global_variables_initializer())
    init_benign_test_score = sess.run(d_model_benign, feed_dict={input_benign: benign_test})
    init_attack_test_score = sess.run(d_model_benign, feed_dict={input_benign: attack_test})
    for e in range(epochs):
        start = time.time()
        np.random.shuffle(benign_train)
        np.random.shuffle(attack_train)
        original_flows.append([])
        generated_flows.append([])
        attack_scores.append([])
        for ii in range(benign_train.shape[0]//batch_size):
            batch_benign = benign_train[ii * batch_size:(ii + 1) * batch_size]
            
            batch_attack = attack_train[ii * batch_size:(ii + 1) * batch_size]
            batch_z = batch_attack[:,:z_size]
            batch_attack_remains = batch_attack[:,z_size:]
            
#             if ii % log_interval == 0:
#                 original_flow = batch_attack
#                 generated_flow = sess.run(g_model, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
#                 attack_score = sess.run(d_model_attack, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
#                 original_flows[-1].append(deepcopy(original_flow[0]))
#                 generated_flows[-1].append(generated_flow[0])
#                 attack_scores[-1].append(attack_score[0][0])
                
            
            # Run optimizers
            _ = sess.run(d_train_opt, feed_dict={input_benign: batch_benign, input_z: batch_z, input_attack_remains: batch_attack_remains})
            _ = sess.run(g_train_opt, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_benign: batch_benign, input_attack_remains: batch_attack_remains})
        train_loss_g = g_loss.eval({input_z: batch_z, input_attack_remains: batch_attack_remains})
            
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g),
              "Time elapsed: {:.4f}".format(time.time() - start)
             )    
        # Save losses to view after training
        losses.append((train_loss_d, train_loss_g))
        
        saver.save(sess, './checkpoints/generator.ckpt')
    print('Total training time:', time.time() - total_start)
    
    final_benign_test_score = sess.run(d_model_benign, feed_dict={input_benign: benign_test})
    final_attack_test_score = sess.run(d_model_benign, feed_dict={input_benign: attack_test})

## Training loss

Here we'll check out the training losses for the generator and discriminator.

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator')
plt.plot(losses.T[1], label='Generator')
plt.title("Training Losses")
plt.legend()

# Test Part

In [ ]:
# TODO: test if discriminator is fooled

In [ ]:
print('Accuracy before training')
print('Discriminator benign test accuracy:', (init_benign_test_score <= 0.5).sum() / benign_test.shape[0])
print('Discriminator attack test accuracy:', (init_attack_test_score > 0.5).sum() / attack_test.shape[0])
print('Discriminator total test accuracy:',
      ((init_benign_test_score <= 0.5).sum() + (init_attack_test_score > 0.5).sum()) / (benign_test.shape[0] + attack_test.shape[0]))

In [ ]:
print('Accuracy after training')
print('Discriminator benign test accuracy:', (final_benign_test_score <= 0.5).sum() / benign_test.shape[0])
print('Discriminator attack test accuracy:', (final_attack_test_score > 0.5).sum() / attack_test.shape[0])
print('Discriminator total test accuracy:',
      ((final_benign_test_score <= 0.5).sum() + (final_attack_test_score > 0.5).sum()) / (benign_test.shape[0] + attack_test.shape[0]))

# Output Visualization

In [ ]:
import math

def split_to_two_nearest_factor(x):
    sqrt_x = int(math.sqrt(x))
    i = sqrt_x
    while x % i != 0:
        i -= 1
    return (i, x // i)
fig_size = split_to_two_nearest_factor(input_size)
import numpy as np
import matplotlib.pyplot as plt

# fooled = 0

# fig=plt.figure(figsize=(fig_size[0] * 5, fig_size[1] * 5))
# columns = 2
# rows = len(original_flows)
# for i in range(rows):
#     original_img = original_flows[i][0]
#     generated_img = generated_flows[i][0]
#     ax_original = fig.add_subplot(rows, columns, i * 2 + 1)
#     ax_original.set_title('Epoch {epoch}'.format(epoch=i + 1))
#     plt.imshow(original_img.reshape(fig_size[0],fig_size[1]), cmap='gray')
#     ax_generated = fig.add_subplot(rows, columns, i * 2 + 2)
#     plt.imshow(generated_img.reshape(fig_size[0],fig_size[1]), cmap='gray')
#     if attack_scores[i][0] <= 0.5:
#         ax_generated.set_title('Fooled:' + str(attack_scores[i][0]))
#         fooled += 1
#     else:
#         ax_generated.set_title('Just close:' + str(attack_scores[i][0]))
# plt.show()
# print('Fooled rate:', fooled / len(original_flows))

# Generate attack flows

In [ ]:
#TODO: generate attack flows for Kaihua's discrimination

# Test

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)

plt.show()

In [ ]:
test_url = "http://download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)

test_dataset = tf.data.TextLineDataset(test_fp)
test_dataset = test_dataset.skip(1)             # skip header row
test_dataset = test_dataset.map(parse_csv)      # parse each row with the funcition created earlier
test_dataset = test_dataset.shuffle(1000)       # randomize
test_dataset = test_dataset.batch(32)           # use the same batch size as the training set

In [ ]:
test_accuracy = tfe.metrics.Accuracy()

for (x, y) in test_dataset:
  prediction = tf.argmax(model(x), axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

In [ ]:
class_ids = ["Iris setosa", "Iris versicolor", "Iris virginica"]

predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  name = class_ids[class_idx]
  print("Example {} prediction: {}".format(i, name))